In [1]:
import pandas as pd

In [2]:
new = pd.read_csv("customer_churn_dataset-testing-master.csv")

In [3]:
new

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...
64369,64370,45,Female,33,12,6,21,Basic,Quarterly,947,14,1
64370,64371,37,Male,6,1,5,22,Standard,Annual,923,9,1
64371,64372,25,Male,39,14,8,30,Premium,Monthly,327,20,1
64372,64373,50,Female,18,19,7,22,Standard,Monthly,540,13,1


In [4]:
new.isnull().sum()

CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [5]:
import h2o
from h2o.automl import H2OAutoML

In [5]:
new['Churn'].value_counts()

0    33881
1    30493
Name: Churn, dtype: int64

In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,21 hours 11 mins
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_lakshmiph_822sg8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,335.6 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
# convert pandas DataFrame into H2O Frame
new_df = h2o.H2OFrame(new)
# Describe  the train h20Frame
new_df.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:64374
Cols:12

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
type,int,int,enum,int,int,int,int,enum,enum,int,int,int
mins,1.0,18.0,,1.0,1.0,0.0,0.0,,,100.0,1.0,0.0
mean,32187.5,41.970982073508566,,31.9948271041102,15.080234256066193,5.4006897194519174,17.13395159536445,,,541.0233790039458,15.498850467580228,0.47368502811694163
maxs,64374.0,65.0,,60.0,30.0,10.0,30.0,,,1000.0,30.0,1.0
sigma,18583.317451413244,13.924911309026198,,17.09823404343624,8.81647006865871,3.114004703906883,8.852211096659238,,,260.87480900825335,8.638436266345833,0.4993109202824684
zeros,0,0,,0,0,4967,1594,,,0,0,33881
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,22.0,Female,25.0,14.0,4.0,27.0,Basic,Monthly,598.0,9.0,1.0
1,2.0,41.0,Female,28.0,28.0,7.0,13.0,Standard,Monthly,584.0,20.0,0.0
2,3.0,47.0,Male,27.0,10.0,2.0,29.0,Premium,Annual,757.0,21.0,0.0


In [8]:
x = new_df.columns
y = 'Churn'
# remove label classvariable from feature variable
x.remove(y)

In [9]:
train, test = new_df.split_frame(ratios = [.8], seed = 1234)

In [10]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [11]:
aml = H2OAutoML(max_runtime_secs=350,
                exclude_algos =['DeepLearning'],
                seed = 1,
                # stopping_metric ='logloss',
                # sort_metric ='logloss',
                balance_classes = False,
)
# train model and record time % time
aml.train(x = x, y = y, training_frame = train, validation_frame=test)

AutoML progress: |
16:16:33.249: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
16:16:33.249: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_25_20230721_161633_model_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    662                662                         171737                 4            4            4             14            16            15.9713

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 3.0312589793374574e-10
RMSE: 1.7410511133615397e-05
LogLoss: 2.6600947247532936e-06
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9992587541118196
       0      1      Error    Rate
-----  -----  -----  -------  -------------
0      27153  0      0        (0.0/27153.0)
1      0      24369  0        (0.0/24369.0)
Total  27153  24369  0        (0.0/51522.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999259     1         199
max f2                       0.999259     1         199
max f0point5                 0.999259     1         199
max accuracy                 0.999259     1         199
max precision                1            1         0
max recall                   0.999259     1         199
max specificity              1            1         0
max absolute_mcc             0.999259     1         199
max min_per_class_accuracy   0.999259     1         199
max mean_per_class_accuracy  0.999259     1         199
max tns                      1            27153     0
max fns                      1            12844     0
max fps                      5.53876e-09  27153     399
max tps                      0.999259     24369     199
max tnr                      1            1         0
max fnr                      1            0.527063  0
max fpr                      5.53876e-09  1         399
max tpr                      0.999259     1         199

Gains/Lift Table: Avg response rate: 47.30 %, avg score: 47.30 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0170413                   1                  2.11424  2.11424            1                1            1                           1                   0.0360294       0.0360294                  111.424  111.424            0.0360294
2        0.0203408                   1                  2.11424  2.11424            1                1            1                           1                   0.00697608      0.0430055                  111.424  111.424            0.0430055
3        0.0349365                   1                  2.11424  2.11424            1                1            1                           1                   0.0308589       0.0738643                  111.424  111.424            0.0738643
4        0.0412057                   1                  2.11424  2.11424            1                1            1                           1                   0.0132545       0.0871189                  111.424  111.424            0.0871189
5        0.0510267                   1                  2.11424  2.11424            1                1            1                           1                   0.0207641    